<a href="https://colab.research.google.com/github/felipeabe/artificial-neural-network/blob/main/trabalhoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#uploading files
from google.colab import files
uploaded = files.upload()

#importing libs
import pandas as pd

ModuleNotFoundError: No module named 'google'

In [ ]:
# Importando o dataset multiclasses
# O conjunto de dados 'obesity.csv' contém informações sobre obesidade em diferentes níveis,
# incluindo fatores relacionados ao estilo de vida, alimentação e transporte.
dfm = pd.read_csv('obesity.csv')

# Renomeando as colunas para melhorar a legibilidade e adequação à análise
# O dicionário abaixo mapeia os nomes antigos das colunas para nomes mais intuitivos.
column_mapping = {
    'family_history_with_overweight': 'family_overweight',  # Histórico familiar de sobrepeso
    'FAVC': 'caloric_food',  # Consumo frequente de alimentos calóricos
    'FCVC': 'eat_vegetables',  # Consumo de vegetais
    'NCP': 'meals_day',  # Número de refeições por dia
    'SMOKE': 'smoke',  # Consumo de cigarros
    'CH2O': 'water_day',  # Consumo de água (litros/dia)
    'SCC': 'track_calories',  # Controle de calorias consumidas
    'FAF': 'physical_activity_week',  # Atividade física semanal (frequência)
    'CALC': 'alcohol',  # Consumo de álcool
    'MTRANS': 'transportation_method',  # Método de transporte utilizado
    'NObeyesdad': 'obesity_level'  # Nível de obesidade (variável dependente)
}
dfm = dfm.rename(columns=column_mapping)

# Removendo colunas desnecessárias para análise
# 'CAEC' (comida entre refeições) e 'TUE' (tempo usado em dispositivos eletrônicos) foram consideradas irrelevantes.
dfm = dfm.drop(columns=['CAEC', 'TUE'])

# Codificando variáveis binárias (sim/não) para valores numéricos (0 e 1)
# Transformação essencial para modelos de aprendizado de máquina que requerem dados numéricos.
binary_mappings = {
    "Gender": {"Female": 0, "Male": 1},
    "family_overweight": {"yes": 1, "no": 0},
    "caloric_food": {"yes": 1, "no": 0},
    "smoke": {"yes": 1, "no": 0},
    "track_calories": {"yes": 1, "no": 0}
}
for col, mapping in binary_mappings.items():
    dfm[col] = dfm[col].map(mapping)

# Codificando os níveis de uso de álcool em valores numéricos
# A escala ordinal (nunca, às vezes, frequentemente, sempre) é transformada para facilitar a análise.
encoding_map = {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
dfm['alcohol'] = dfm['alcohol'].map(encoding_map)

# Codificando o método de transporte em categorias relevantes
# Combinações foram realizadas para agrupar modos similares (ex.: 'Bike' e 'Walking' como 0).
custom_mapping = {
    "Walking": 0,
    "Bike": 0,
    "Motorbike": 1,
    "Automobile": 1,
    "Public_Transportation": 1
}
dfm['transportation_method'] = dfm['transportation_method'].map(custom_mapping)

# Codificando os níveis de obesidade em valores numéricos
# A variável dependente é ordinal e vai de 0 (peso insuficiente) a 6 (obesidade tipo III).
obesity_mapping = {
    "Insufficient_Weight": 0,
    "Normal_Weight": 1,
    "Overweight_Level_I": 2,
    "Overweight_Level_II": 3,
    "Obesity_Type_I": 4,
    "Obesity_Type_II": 5,
    "Obesity_Type_III": 6
}
dfm['obesity_level'] = dfm['obesity_level'].map(obesity_mapping)

# Normalizando colunas contínuas (Altura, Peso, Idade) para valores entre 0 e 1
# A normalização é importante para evitar que escalas distintas afetem a performance dos modelos.
columns_to_normalize = ["Height", "Weight", "Age"]
dfm[columns_to_normalize] = dfm[columns_to_normalize].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)

# Dividindo o conjunto de dados em treino e teste
# Usamos 80% dos dados para treino e 20% para teste, mantendo a aleatoriedade dos registros.
split_point = int(0.8 * len(dfm))
train_multiclass = dfm[:split_point]  # Conjunto de treino
test_multiclass = dfm[split_point:]  # Conjunto de teste

# Visualizando as 10 primeiras linhas do dataset processado
dfm.head(10)


,Gender,Age,Height,Weight,family_overweight,caloric_food,eat_vegetables,meals_day,smoke,water_day,track_calories,physical_activity_week,alcohol,transportation_method,obesity_level
0,0,0.149,0.321,0.187,1,0,2.0,3.0,0,2.0,0,0.0,0,1,1
1,0,0.149,0.132,0.127,1,0,3.0,3.0,1,3.0,1,3.0,1,1,1
2,1,0.191,0.660,0.284,1,0,2.0,3.0,0,2.0,0,2.0,2,1,1
3,1,0.277,0.660,0.358,0,0,3.0,3.0,0,2.0,0,2.0,2,0,2
4,1,0.170,0.623,0.379,0,0,2.0,1.0,0,2.0,0,0.0,1,1,3
5,1,0.319,0.321,0.104,0,1,2.0,3.0,0,2.0,0,0.0,1,1,1
6,0,0.191,0.094,0.119,1,1,3.0,3.0,0,2.0,0,1.0,1,1,1
7,1,0.170,0.358,0.104,0,0,2.0,3.0,0,2.0,0,3.0,1,1,1
8,1,0.213,0.623,0.187,1,1,3.0,3.0,0,2.0,0,1.0,2,1,1
9,1,0.170,0.509,0.216,1,1,2.0,3.0,0,2.0,0,1.0,0,1,1


In [ ]:
# Importando o dataset de regressão
# O conjunto de dados 'houses.csv' contém informações sobre imóveis, como preço, área útil e outros atributos.
dfr = pd.read_csv('houses.csv')

# Removendo colunas desnecessárias para análise
# As colunas 'renovated' (renovação) e 'quartile_zone' (zona por quartil) foram excluídas por serem irrelevantes para a tarefa de regressão.
dfr = dfr.drop(columns=['renovated', 'quartile_zone'])

# Codificando valores booleanos (True/False) em numéricos (1/0)
# Transformação essencial para modelos de aprendizado de máquina que não lidam diretamente com valores booleanos.
dfr = dfr.replace({True: 1, False: 0})

# Extraindo o ano a partir da coluna de data
# A coluna 'date' foi convertida para o formato datetime, e apenas o ano foi extraído para representar a idade do imóvel.
dfr['date'] = pd.to_datetime(dfr['date'])  # Conversão para formato datetime
dfr['date'] = dfr['date'].dt.year  # Extração do ano

# Normalizando colunas contínuas para valores entre 0 e 1
# A normalização das colunas 'price' (preço) e 'living_in_m2' (área útil) é essencial para manter escalas consistentes entre os atributos.
columns_to_normalize = ["price", "living_in_m2"]
dfr[columns_to_normalize] = dfr[columns_to_normalize].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)

# Dividindo o conjunto de dados em treino e teste
# Utilizamos 80% dos dados para o treino e 20% para teste, preservando a aleatoriedade dos registros.
split_point3 = int(0.8 * len(dfr))
train_regression = dfr[:split_point3]  # Conjunto de treino
test_regression = dfr[split_point3:]  # Conjunto de teste

# Visualizando as 10 primeiras linhas do conjunto de treino
train_regression.head(10)
    

<ipython-input-6-1c407e60d625>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfr = dfr.replace({True: 1, False: 0})


,date,price,bedrooms,grade,has_basement,living_in_m2,nice_view,perfect_condition,real_bathrooms,has_lavatory,single_floor,month
0,2014,0.215,2,2,1,0.261,0,0,2,1,0,5
1,2014,0.213,2,2,0,0.166,0,0,1,1,0,11
2,2014,0.223,2,2,0,0.201,1,0,1,0,1,12
3,2015,0.171,2,3,0,0.299,0,0,2,1,1,2
4,2015,0.576,3,2,1,0.856,0,0,3,0,0,1
5,2015,0.339,1,2,1,0.103,1,0,1,0,1,3
6,2014,0.149,2,2,1,0.253,0,0,1,1,1,6
7,2014,0.143,1,1,0,0.264,0,0,1,0,1,10
8,2014,0.116,2,2,0,0.166,0,0,1,1,1,7
9,2014,0.432,2,3,0,0.204,0,0,2,1,0,9


In [ ]:
# Importando bibliotecas necessárias
# Bibliotecas como pandas, numpy e sklearn são utilizadas para processamento de dados e divisão do conjunto de dados.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Carregar e processar o conjunto de dados
# O conjunto 'alzheimer.csv' contém informações sobre pacientes e seus diagnósticos relacionados à doença de Alzheimer.
alzheimer_path = "alzheimer.csv"
alzheimer_data = pd.read_csv(alzheimer_path)

# Visualizar os primeiros registros do conjunto de dados
alzheimer_data.head()

# Remover colunas irrelevantes
# As colunas 'PatientID' e 'DoctorInCharge' foram removidas por não influenciarem diretamente no diagnóstico.
alzheimer_cleaned = alzheimer_data.drop(columns=["PatientID", "DoctorInCharge"])

# Separar variáveis categóricas e numéricas
# Variáveis categóricas como gênero, etnia e nível educacional são identificadas para codificação.
categorical_columns = ["Gender", "Ethnicity", "EducationLevel"]
numerical_columns = [col for col in alzheimer_cleaned.columns if col not in categorical_columns + ["Diagnosis"]]

# Codificar variáveis categóricas
# OneHotEncoder é utilizado para converter variáveis categóricas em representações numéricas binárias.
encoder = OneHotEncoder(drop="first", sparse_output=False)  # Evitar multicolinearidade
categorical_encoded = encoder.fit_transform(alzheimer_cleaned[categorical_columns])
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Normalizar variáveis numéricas
# As variáveis numéricas são escalonadas para manter os valores em uma escala padrão (média 0, desvio padrão 1).
scaler = StandardScaler()
numerical_scaled = scaler.fit_transform(alzheimer_cleaned[numerical_columns])
numerical_scaled_df = pd.DataFrame(numerical_scaled, columns=numerical_columns)

# Combinar dados processados e variável alvo
# Dados normalizados e codificados são combinados em um único DataFrame para análise posterior.
processed_data = pd.concat([numerical_scaled_df, categorical_encoded_df], axis=1)
processed_data["Diagnosis"] = alzheimer_cleaned["Diagnosis"]

# Dividir em conjuntos de treino e teste
# O conjunto de dados é dividido em 80% para treino e 20% para teste, garantindo dados independentes para validação.
X = processed_data.drop(columns=["Diagnosis"])
y = processed_data["Diagnosis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Funções de ativação e suas derivadas
# Funções de ativação são essenciais para redes neurais, introduzindo não-linearidades no modelo.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return np.tanh(x)

def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def tanh_derivative(x):
    return 1 - np.tanh(x)**2

# Funções de perda
# A função de perda mede a discrepância entre as previsões do modelo e os valores reais.
def cross_entropy_loss(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))

def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Implementação da Rede Neural para Classificação Binária
class NeuralNetworkBinary:
    def __init__(self, input_size, hidden_size, activation="relu", loss="cross_entropy"):
        # Inicializa os pesos e biases
        self.weights_input_hidden = np.random.randn(input_size, hidden_size) * 0.01
        self.bias_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.randn(hidden_size, 1) * 0.01
        self.bias_output = np.zeros((1, 1))

        # Configurar função de ativação
        self.activation = activation
        self.loss = loss

        if activation == "relu":
            self.activation_func = relu
            self.activation_derivative = relu_derivative
        elif activation == "sigmoid":
            self.activation_func = sigmoid
            self.activation_derivative = sigmoid_derivative
        elif activation == "tanh":
            self.activation_func = tanh
            self.activation_derivative = tanh_derivative
        else:
            raise ValueError("Ativação inválida! Escolha entre 'relu', 'sigmoid' ou 'tanh'.")

        # Configurar função de perda
        if loss == "cross_entropy":
            self.loss_func = cross_entropy_loss
        elif loss == "mse":
            self.loss_func = mse_loss
        else:
            raise ValueError("Função de perda inválida! Escolha entre 'cross_entropy' ou 'mse'.")

    def forward(self, X):
        # Propagação para frente
        self.z1 = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.a1 = self.activation_func(self.z1)
        self.z2 = np.dot(self.a1, self.weights_hidden_output) + self.bias_output
        self.a2 = sigmoid(self.z2)  # A saída final é sempre sigmoid (probabilidade)
        return self.a2

    def backward(self, X, y, learning_rate):
        # Retropropagação para ajuste dos pesos
        m = X.shape[0]
        output_error = self.a2 - y  # Erro da saída

        # Gradientes para a camada de saída
        dw2 = np.dot(self.a1.T, output_error) / m
        db2 = np.sum(output_error, axis=0, keepdims=True) / m

        # Gradientes para a camada oculta
        hidden_error = np.dot(output_error, self.weights_hidden_output.T) * self.activation_derivative(self.z1)
        dw1 = np.dot(X.T, hidden_error) / m
        db1 = np.sum(hidden_error, axis=0, keepdims=True) / m

        # Atualizar pesos e biases
        self.weights_hidden_output -= learning_rate * dw2
        self.bias_output -= learning_rate * db2
        self.weights_input_hidden -= learning_rate * dw1
        self.bias_hidden -= learning_rate * db1

    def compute_loss(self, y_true, y_pred):
        return self.loss_func(y_true, y_pred)

# Conversão dos conjuntos de treino e teste para numpy
# Necessário para compatibilidade com a implementação manual da rede neural.
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy().reshape(-1, 1)
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy().reshape(-1, 1)

# Configurações da rede neural
input_size = X_train_np.shape[1]  # Número de features
hidden_size = 10  # Neurônios na camada oculta
learning_rate = 0.02
epochs = 1000

# Treinamento com MSE
nn_binary = NeuralNetworkBinary(input_size=input_size, hidden_size=hidden_size, activation="tanh", loss="mse")
losses = []
for epoch in range(epochs):
    y_pred = nn_binary.forward(X_train_np)
    loss = nn_binary.compute_loss(y_train_np, y_pred)
    losses.append(loss)
    nn_binary.backward(X_train_np, y_train_np, learning_rate)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss:.4f}")

# Avaliação com MSE
y_test_pred = nn_binary.forward(X_test_np)
test_accuracy = np.mean((y_test_pred > 0.5) == y_test_np)
print(f"Acurácia no conjunto de teste com MSE: {test_accuracy * 100:.2f}%")

# Repetição para Cross Entropy
nn_binary = NeuralNetworkBinary(input_size=input_size, hidden_size=hidden_size, activation="tanh", loss="cross_entropy")
losses = []
for epoch in range(epochs):
    y_pred = nn_binary.forward(X_train_np)
    loss = nn_binary.compute_loss(y_train_np, y_pred)
    losses.append(loss)
    nn_binary.backward(X_train_np, y_train_np, learning_rate)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss:.4f}")

# Avaliação com Cross Entropy
y_test_pred = nn_binary.forward(X_test_np)
test_accuracy = np.mean((y_test_pred > 0.5) == y_test_np)
print(f"Acurácia no conjunto de teste com Cross Entropy: {test_accuracy * 100:.2f}%")


Epoch 0/1000, Loss: 0.2501
Epoch 100/1000, Loss: 0.2363
Epoch 200/1000, Loss: 0.2309
Epoch 300/1000, Loss: 0.2278
Epoch 400/1000, Loss: 0.2231
Epoch 500/1000, Loss: 0.2120
Epoch 600/1000, Loss: 0.1911
Epoch 700/1000, Loss: 0.1649
Epoch 800/1000, Loss: 0.1426
Epoch 900/1000, Loss: 0.1280
Acurácia no conjunto de teste utilizando mse: 83.02% 

Epoch 0/1000, Loss: 0.6932
Epoch 100/1000, Loss: 0.6652
Epoch 200/1000, Loss: 0.6531
Epoch 300/1000, Loss: 0.6428
Epoch 400/1000, Loss: 0.6210
Epoch 500/1000, Loss: 0.5708
Epoch 600/1000, Loss: 0.4967
Epoch 700/1000, Loss: 0.4350
Epoch 800/1000, Loss: 0.3992
Epoch 900/1000, Loss: 0.3806
Acurácia no conjunto de teste utilizando cross_entropy: 82.79% 

